<a href="https://colab.research.google.com/github/Yanina-Kutovaya/GNN/blob/main/notebooks/train_test_val_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Функция train_test_val_split для обучения графовых моделей

## Что делает функция:
- принимает объект ```Data``` из PyTorch Geometric,
- разбивает индексы на обучающую, валидационную и тестовую выборки  с учетом баланса классов (стратификация) без использования ```sklearn```,
- создаёт соответствующие булевы маски и добавляет их в объект ```Data```,
- позволяет задать seed для воспроизводимости,
- поддерживает логирование,
- осуществляет подсчет количества узлов каждого класса в ```train```, ```val```, ```test``` и сохраняет эту статистику в CSV-файл для отслеживания баланса классов после стратификации,
- дает возможность указать путь для сохранения файла,
- проводит проверку распределения классов и логирование общего баланса классов.
- предупреждает, если какой-то класс составляет больше, чем ```threshold``` от всех данных (доминирующий класс),
- предупреждает, если какой-то класс имеет меньше, чем ```min_samples``` примеров (малочисленный класс ).



## 1. Установка зависимостей

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [2]:
install = True
if install:
  # 1. Установка совместимых версий PyTorch и PyG
  !pip install -q torch==2.3.0+cu121 torchvision==0.18.0+cu121 torchaudio==2.3.0+cu121 --extra-index-url https://download.pytorch.org/whl/cu121

  # 2. Установка зависимостей PyG для CUDA 12.1+ (совместимо с 12.5)
  !pip install -q pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.3.0+cu121.html --no-cache-dir

  # 3. Установка PyTorch Geometric
  !pip install -q torch-geometric==2.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.0/781.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 111.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

__Проверка окружения:__

In [3]:
import torch
print(f"PyTorch: {torch.__version__}")          # Должно быть 2.3.0+cu121
print(f"CUDA: {torch.version.cuda}")            # Должно быть 12.1+
print(f"Available: {torch.cuda.is_available()}")# Должно быть True

PyTorch: 2.3.0+cu121
CUDA: 12.1
Available: True


## 2. Функция train_test_val_split

In [4]:
import numpy as np
import torch
import logging
import csv
import warnings
from torch_geometric.data import Data
from collections import defaultdict

# Настройка логгера
logger = logging.getLogger(__name__)
if not logger.hasHandlers():
    handler = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)

# Настройки для предупреждений
warnings.simplefilter('always', UserWarning)


def train_test_val_split(data: Data,
                         train_ratio: float = 0.6,
                         val_ratio: float = 0.2,
                         test_ratio: float = 0.2,
                         seed: int = 42,
                         save_stats_path: str = None,
                         imbalance_threshold: float = 0.8,
                         min_class_samples: int = 5) -> Data:
    """
    Разбивает данные на обучающую, валидационную и тестовую выборки
    с **стратификацией по классам**, создаёт булевы маски и добавляет их в объект Data.

    Параметры:
        data (Data): Объект данных PyG с полем data.y (метки классов).
        train_ratio (float): Доля обучающей выборки.
        val_ratio (float): Доля валидационной выборки.
        test_ratio (float): Доля тестовой выборки.
        seed (int): Seed для воспроизводимости.
        save_stats_path (str | None): Путь для сохранения статистики распределения классов в формате CSV.
        imbalance_threshold (float): Доля, при превышении которой класс считается доминирующим (выдаётся warning).
        min_class_samples (int): Минимальное количество элементов в классе, ниже которого будет warning.

    Возвращает:
        Data: Объект Data с добавленными масками train_mask, val_mask, test_mask.
    """
    logger.info("Начало разделения данных на train/val/test с стратификацией")

    # Проверка корректности соотношений
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, \
        "Сумма долей train, val и test должна быть равна 1."
    logger.debug(f"Доли выборок: train={train_ratio}, val={val_ratio}, test={test_ratio}")

    # Устанавливаем фиксированный seed для воспроизводимости
    np.random.seed(seed)
    logger.debug(f"Установлен seed={seed} для воспроизводимости")

    # Общее количество узлов
    num_nodes = data.num_nodes
    logger.info(f"Общее количество узлов: {num_nodes}")

    # Проверяем наличие меток классов
    if not hasattr(data, 'y'):
        raise ValueError("Данные не содержат меток классов (data.y). "
                         "Стратификация невозможна.")
    labels = data.y.numpy()

    # Словарь: класс -> список индексов узлов этого класса
    class_indices = {}
    for idx in range(num_nodes):
        label = labels[idx]
        if label not in class_indices:
            class_indices[label] = []
        class_indices[label].append(idx)

    logger.debug(f"Количество уникальных классов: {len(class_indices)}")
    logger.debug("Распределение классов:")
    for cls, indices in class_indices.items():
        logger.debug(f"Класс {cls}: {len(indices)} узлов")

    # --- Проверка на дисбаланс классов ---
    total = len(labels)
    for cls, indices in class_indices.items():
        count = len(indices)
        ratio = count / total

        if ratio > imbalance_threshold:
            warn_msg = f"Класс {cls} составляет {ratio:.2%} от всех данных — это больше порога {imbalance_threshold}. Возможен дисбаланс!"
            logger.warning(warn_msg)
            warnings.warn(warn_msg, UserWarning)

        if count < min_class_samples:
            warn_msg = f"Класс {cls} содержит всего {count} образцов — это меньше минимального порога {min_class_samples}."
            logger.warning(warn_msg)
            warnings.warn(warn_msg, UserWarning)

    # Для каждого класса разбиваем индексы пропорционально
    train_idx, val_idx, test_idx = [], [], []

    for cls, indices in class_indices.items():
        num_total = len(indices)
        num_train = int(train_ratio * num_total)
        num_val = int(val_ratio * num_total)

        permuted = np.random.permutation(indices)
        train_idx.extend(permuted[:num_train])
        val_idx.extend(permuted[num_train:num_train + num_val])
        test_idx.extend(permuted[num_train + num_val:])

    logger.debug(f"Количество элементов после стратификации: "
                 f"train={len(train_idx)}, val={len(val_idx)}, test={len(test_idx)}")

    # Преобразуем в массивы numpy
    train_idx = np.array(train_idx)
    val_idx = np.array(val_idx)
    test_idx = np.array(test_idx)

    # Создаем булевы маски
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    # Заполняем маски
    train_mask[train_idx] = True
    val_mask[val_idx] = True
    test_mask[test_idx] = True

    # Добавляем маски в объект Data
    data.train_mask = train_mask
    data.val_mask = val_mask
    data.test_mask = test_mask

    logger.info("Маски успешно созданы и добавлены в объект Data")
    logger.info(f"Размеры масок: train={train_mask.sum().item()}, "
                f"val={val_mask.sum().item()}, "
                f"test={test_mask.sum().item()}")

    # --- Сохранение статистики ---
    if save_stats_path:
        stats = []

        for cls in sorted(class_indices.keys()):
            total_cls = len(class_indices[cls])
            train_count = sum([1 for idx in class_indices[cls] if train_mask[idx]])
            val_count = sum([1 for idx in class_indices[cls] if val_mask[idx]])
            test_count = sum([1 for idx in class_indices[cls] if test_mask[idx]])

            stats.append({
                'class': cls,
                'total': total_cls,
                'train': train_count,
                'val': val_count,
                'test': test_count,
                'train_ratio': round(train_count / total_cls, 2) if total_cls else 0,
                'val_ratio': round(val_count / total_cls, 2) if total_cls else 0,
                'test_ratio': round(test_count / total_cls, 2) if total_cls else 0,
            })

        with open(save_stats_path, mode='w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=stats[0].keys())
            writer.writeheader()
            writer.writerows(stats)

        logger.info(f"Статистика распределения классов сохранена в файл: {save_stats_path}")

    return data

## 3. Пример использования:

### 3.1. Загрузка данных

In [5]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]

Processing...
Done!


### 3.2. Пример вызова функции

In [6]:
data = train_test_val_split(
    data,
    train_ratio=0.6,
    val_ratio=0.2,
    test_ratio=0.2,
    seed=42,
    save_stats_path="class_distribution.csv",
    imbalance_threshold=0.7,
    min_class_samples=3
)

In [7]:
print("Train mask size:", data.train_mask.sum().item())
print("Val mask size:", data.val_mask.sum().item())
print("Test mask size:", data.test_mask.sum().item())

Train mask size: 1621
Val mask size: 539
Test mask size: 548


In [8]:
import pandas as pd

pd.read_csv('/content/class_distribution.csv')

,class,total,train,val,test,train_ratio,val_ratio,test_ratio
0,0,351,210,70,71,0.6,0.2,0.2
1,1,217,130,43,44,0.6,0.2,0.2
2,2,418,250,83,85,0.6,0.2,0.2
3,3,818,490,163,165,0.6,0.2,0.2
4,4,426,255,85,86,0.6,0.2,0.2
5,5,298,178,59,61,0.6,0.2,0.2
6,6,180,108,36,36,0.6,0.2,0.2


### 3.3. Пример вывода предупреждений:

- WARNING:```__main__```:Класс 0 составляет 82.00% от всех данных — это больше порога 0.7. Возможен дисбаланс!
- WARNING:```__main__```:Класс 2 содержит всего 2 образца — это меньше минимального порога 3.